# Resiliência da paisagem

## Importando pacotes e inicializando *geemap*

In [1]:
import os
import ee
import geemap
import geemap.colormaps as cm

In [2]:
geemap.ee_initialize()

A resiliência da paisagem é classificada pela heterogeneidade da paisagem e conectividade local [@anderson_resilient_2016]. Modificamos a metodologia original de média das métricas pela classificação pelos quantis das duas variáveis. Classificamos as métricas por quantis e atribuímos classes para as combinações de quantis de heterogeneidade da paisagem e conectividade local de cada célula. Por exemplo, células com alta heterogeneidade local (quantil superior)  e alta conectividade local (quantil superior) são áreas com alta resiliência. Por outro lado, locais com baixa conectividade local e heterogeneidade da paisagem possuem baixa resiliência. Nas classes intermediárias estão as células com alta heterogeneidade da paisagem e baixa conectividadel local, e as células com alta conectividade local e baixa heterogeneidade da paisagem. 

## Base de dados

Os dados utilizados para calcular a resiliência da paisagem são os mapa de heterogeneidade da paisagem e a conectividade local criados anteriormente.

## Códigos para a criação da resiliência da paisagem

As análises foram rodadas no *Google Earth Engine* [@gorelick_google_2017], devido a demanda computacional do projeto, usando o pacote **geemap** [@wu_geemap_2020] em *Python* [@python_software_foundation_python_2023] como interface pela facilidade na documentação e reprodutividade das análises. O JupyterNotebook para a reprodução das análises pode ser encontrado em [https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/7_resilience.ipynb](https://github.com/Resiliencia-climatica-Brasil/diversity-resilience-python/blob/master/jupyternotebook/7_resilience.ipynb).

Primeiro, importamos o polígono de biomas do Brasil para extraímos as coordenadas geográficas mínimas e máximas. Extraímos também a escala do modelo digital de elevação que usamos nas análises anteriores.

In [13]:
# Importando o polígono de bioma para definir as 
# coordenadas máximas e mínimas do Brasil

regiao = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

# ModeloDigital de Elevação para extrair a resolução
DEM = ee.Image("MERIT/DEM/v1_0_3")

# função para extrair as bordas dos polígonos
def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
regiao_box = regiao.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

# Resolução das análises
escala = DEM.projection().nominalScale()

Importamos os mapas de heterogeneidade da paisagem e conectividade local.

In [14]:
# modifique os endereços para os do seu projeto

local_connectedness = ee.Image("projects/ee-lucasljardim9/assets/local_connectedness")

landscape_heterogeneity = ee.Image("projects/ee-lucasljardim9/assets/landscape_heterogeneity")

Calculamos os quantis de 25, 50, 75% da heterogeneidade da paisagem e conectividade local.

In [15]:
# Heterogeneidade da paisagem
quartiles_Div = landscape_heterogeneity.reduceRegion(**{
    'reducer': ee.Reducer.percentile([25, 50, 75]), 
    'scale': escala,
    'maxPixels':1e13,
    'bestEffort': False
  })

# Extraímos os valores do quantis
div_25 = quartiles_Div.getNumber('z_p25')
div_50 = quartiles_Div.getNumber('z_p50')
div_75 = quartiles_Div.getNumber('z_p75')

# Conectividade local
quartiles_Con = local_connectedness.reduceRegion(**{
    'reducer': ee.Reducer.percentile([25, 50, 75]), 
    'scale': escala,
    'maxPixels':1e13,
    'bestEffort': False
  })

# Extraímos os valores do quantis
con_25 = quartiles_Con.getNumber('b1_p25')
con_50 = quartiles_Con.getNumber('b1_p50')
con_75 = quartiles_Con.getNumber('b1_p75')

Classificamos a heterogeneidade da paisagem e conectividade local pelos quantis.

In [16]:
class_div = (landscape_heterogeneity
             .where(landscape_heterogeneity.lt(div_25), 1)  
             .where(landscape_heterogeneity.gte(div_25).And(landscape_heterogeneity.lt(div_50)), 2)
             .where(landscape_heterogeneity.gte(div_50).And(landscape_heterogeneity.lt(div_75)), 3)
             .where(landscape_heterogeneity.gte(div_75).And(landscape_heterogeneity), 4))


class_con = (local_connectedness
             .where(local_connectedness.lt(con_25),10)
             .where(local_connectedness.gte(con_25).And(local_connectedness.lt(con_50)), 20)
             .where(local_connectedness.gte(con_50).And(local_connectedness.lt(con_75)),30)
             .where(local_connectedness.gte(con_75).And(local_connectedness), 40))

Somamos os mapas de heterogeneidade da paisagem e conectividade local classificados para atribuirmos as classes de resiliência. 

In [17]:
resilience = class_div.add(class_con)

Exportamos o mapa de resiliência como um *asset* no *Google Earth Engine*.

In [23]:
# Exporte o raster de resiliência
# modifique o assetId para o do seu projeto

assetId = "projects/ee-lucasljardim9/assets/resilience"

geemap.ee_export_image_to_asset(
    resilience, 
    description='resilience', 
    assetId=assetId, 
    region=regiao_box, 
    scale=escala,
    maxPixels=1e13
)

O dado pode ser visualizado abaixo:

In [7]:
# Paleta de cores

sld_intervals =(
  '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="False">' + \
      '<ColorMapEntry color="#d3d3d3" quantity="11" label="11"/>' + \
      '<ColorMapEntry color="#95bdd5" quantity="12" label="12"/>' + \
      '<ColorMapEntry color="#52a6d7" quantity="13" label="13"/>' + \
      '<ColorMapEntry color="#0088d9" quantity="14" label="14"/>' + \
      '<ColorMapEntry color="#d5ce9c" quantity="21" label="21"/>' + \
      '<ColorMapEntry color="#96b89e" quantity="22" label="22"/>' + \
      '<ColorMapEntry color="#53a19f" quantity="23" label="23"/>' + \
      '<ColorMapEntry color="#0085a1" quantity="24" label="24"/>' + \
      '<ColorMapEntry color="#d7c759" quantity="31" label="31"/>' + \
      '<ColorMapEntry color="#98b35a" quantity="32" label="32"/>' + \
      '<ColorMapEntry color="#539c5b" quantity="33" label="33"/>' + \
      '<ColorMapEntry color="#00815c" quantity="34" label="34"/>' + \
      '<ColorMapEntry color="#d9be00" quantity="41" label="41"/>' + \
      '<ColorMapEntry color="#9aaa00" quantity="42" label="42"/>' + \
      '<ColorMapEntry color="#549500" quantity="43" label="43"/>' + \
      '<ColorMapEntry color="#007b00" quantity="44" label="44"/>' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>')

# Centralize o mapa

Map = geemap.Map(center=(-11.75, -51.52), zoom=4)

# Importe os assets

resilience = ee.Image("projects/ee-lucasljardim9/assets/resilience")


# Plote o mapa

Map.addLayer(resilience.sldStyle(sld_intervals)) 


Map

Map(center=[-11.75, -51.52], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…